<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/tensorflow-js/tensorflow_js_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook needs a GPU resource. Check the GPU assigned.

In [0]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-bf426fbe-a855-7a0d-5aaf-2c851ce6615e)


In [0]:
#@title ### Install nodejs v13.x
!sudo apt-get update -y -qq 
!sudo apt-get upgrade -y -qq 
!curl -sL https://deb.nodesource.com/setup_13.x | sudo bash -
!sudo apt-get install -y -qq nodejs
!node --version

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 54.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../base-files_10.1ubuntu2.8_amd64.deb ...
Unpacking base-files (10.1ubuntu2.8) over (10.1ubuntu2.7) ...
Setting up base-files (10.1ubuntu2.8) ...
Installing new version of config file /etc/issue ...
Installing new version of config file /etc/issue.net ...
Installing new version of config file /etc/lsb-release ...
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../bsdutils_1%3a2.31.1-0.4ubuntu3.6_amd64.deb .

In [0]:
!npm --version

6.14.4


In [0]:
#@title ## Setup javascript kernel
#@markdown We use ``ijavascript`` to enable a javascript kernel inside Colab. 

#@markdown ---
#@markdown **Important:** Wait until this cell's run completes and reload your 
#@markdown environment pressing ``F5``.

#@markdown ---

!npm install - g npm@latest
!npm cache verify
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global
!jupyter-kernelspec list

npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN content@1.0.0 No description
npm WARN content@1.0.0 No repository field.

+ -@0.0.1
+ g@2.0.1
+ npm@6.14.4
added 437 packages from 870 contributors and audited 12054 packages in 9.89s

1 package is looking for funding
  run `npm fund` for details

found 0 vulnerabilities

Cache verified and compressed (~/.npm/_cacache):
Content verified: 82 (21057962 bytes)
Index entries: 85
Finished in 0.752s
/usr/bin/ijs -> /usr/lib/node_modules/ijavascript/bin/ijavascript.js
/usr/bin/ijsconsole -> /usr/lib/node_modules/ijavascript/bin/ijsconsole.js
/usr/bin/ijsinstall -> /usr/lib/node_modules/ijavascript/bin/ijsinstall.js
/usr/bin/ijskernel -> /usr/lib/node_modules/ijavascript/lib/kernel.js
/usr/bin/ijsnotebook -> /usr/lib/node_modules/ijavascript/bin/ijsnotebook.js

> zeromq@5.2.0 install /usr/lib/node_modules/ijavascript/node_modules/zeromq
> node scripts/prebuild-install.js || (node scripts/preinstall.js && 

# Training

In [3]:
#@title ## Install TensorFlow.js
#@markdown If you run into any "SyntaxError: invalid syntax" issue, you have
#@markdown probably not reloaded this notebook. Press ``F5`` and try to 
#@markdown execute this cell again.
 
var { spawn } = require('child_process');
var sh = (cmd) => { 
    $$.async();
    var sp = spawn(cmd, { cwd: process.cwd(), stdio: 'pipe', shell: true, encoding: 'utf-8' });
    sp.stdout.on('data', data => console.log(data.toString()));
    sp.stderr.on('data', data => console.error(data.toString()));
    sp.on('close', () => $$.done());
};
var run_async = async (pf) => {
  $$.async();
  await pf();
  $$.done();
};
// sh('npm init -y -q');
// sh('npm install @tensorflow/tfjs-node-gpu -q')
sh('npm install')


> @tensorflow/tfjs-node-gpu@1.7.2 install /content/node_modules/@tensorflow/tfjs-node-gpu
> node scripts/install.js gpu download


GPU-linux-1.7.2.tar.gz



* Downloading libtensorflow



* Building TensorFlow Node.js bindings

npm 
WARN content@1.0.0 No description
npm WARN content@1.0.0 No repository field.





added 93 packages from 99 contributors and audited 12220 packages in 19.565s


1 package is looking for funding
  run `npm fund` for details


found 0 vulnerabilities




In [0]:
sh('NODE_OPTIONS=1')

In [0]:
sh('echo $NODE_OPTIONS')

In [0]:
sh('npm install');


> @tensorflow/tfjs-node-gpu@1.7.2 install /content/node_modules/@tensorflow/tfjs-node-gpu
> node scripts/install.js gpu download


GPU-linux-1.7.2.tar.gz



* Downloading libtensorflow



* Building TensorFlow Node.js bindings

npm 
WARN content@1.0.0 No description
npm WARN content@1.0.0 No repository field.





+ @tensorflow/tfjs-node-gpu@1.7.2
added 93 packages from 99 contributors and audited 12220 packages in 15.343s


1 package is looking for funding
  run `npm fund` for details


found 0 vulnerabilities




In [13]:
var tf = require('@tensorflow/tfjs-node-gpu');
console.log('GPU enabled:', tf.backend().isUsingGpuDevice);

GPU enabled: true


In [0]:
var tf = require('@tensorflow/tfjs-node-gpu');
var assert = require('assert');
var fs = require('fs');
var https = require('https');
var util = require('util');
var zlib = require('zlib');

var readFile = util.promisify(fs.readFile);

// MNIST data constants:
var BASE_URL = 'https://storage.googleapis.com/cvdf-datasets/mnist/';
var TRAIN_IMAGES_FILE = 'train-images-idx3-ubyte';
var TRAIN_LABELS_FILE = 'train-labels-idx1-ubyte';
var TEST_IMAGES_FILE = 't10k-images-idx3-ubyte';
var TEST_LABELS_FILE = 't10k-labels-idx1-ubyte';
var IMAGE_HEADER_MAGIC_NUM = 2051;
var IMAGE_HEADER_BYTES = 16;
var IMAGE_HEIGHT = 28;
var IMAGE_WIDTH = 28;
var IMAGE_FLAT_SIZE = IMAGE_HEIGHT * IMAGE_WIDTH;
var LABEL_HEADER_MAGIC_NUM = 2049;
var LABEL_HEADER_BYTES = 8;
var LABEL_RECORD_BYTE = 1;
var LABEL_FLAT_SIZE = 10;

// Downloads a test file only once and returns the buffer for the file.
async function fetchOnceAndSaveToDiskWithBuffer(filename) {
  return new Promise(resolve => {
    var url = `${BASE_URL}${filename}.gz`;
    if (fs.existsSync(filename)) {
      resolve(readFile(filename));
      return;
    }
    var file = fs.createWriteStream(filename);
    console.log(`  * Downloading from: ${url}`);
    https.get(url, response => {
      var unzip = zlib.createGunzip();
      response.pipe(unzip).pipe(file);
      unzip.on('end', () => {
        resolve(readFile(filename));
      });
    });
  });
}

function loadHeaderValues(buffer, headerLength) {
  var headerValues = [];
  for (var i = 0; i < headerLength / 4; i++) {
    // Header data is stored in-order (aka big-endian)
    headerValues[i] = buffer.readUInt32BE(i * 4);
  }
  return headerValues;
}

async function loadImages(filename) {
  var buffer = await fetchOnceAndSaveToDiskWithBuffer(filename);

  var headerBytes = IMAGE_HEADER_BYTES;
  var recordBytes = IMAGE_HEIGHT * IMAGE_WIDTH;

  var headerValues = loadHeaderValues(buffer, headerBytes);
  assert.equal(headerValues[0], IMAGE_HEADER_MAGIC_NUM);
  assert.equal(headerValues[2], IMAGE_HEIGHT);
  assert.equal(headerValues[3], IMAGE_WIDTH);

  var images = [];
  var index = headerBytes;
  while (index < buffer.byteLength) {
    var array = new Float32Array(recordBytes);
    for (var i = 0; i < recordBytes; i++) {
      // Normalize the pixel values into the 0-1 interval, from
      // the original 0-255 interval.
      array[i] = buffer.readUInt8(index++) / 255;
    }
    images.push(array);
  }

  assert.equal(images.length, headerValues[1]);
  return images;
}

async function loadLabels(filename) {
  var buffer = await fetchOnceAndSaveToDiskWithBuffer(filename);

  var headerBytes = LABEL_HEADER_BYTES;
  var recordBytes = LABEL_RECORD_BYTE;

  var headerValues = loadHeaderValues(buffer, headerBytes);
  assert.equal(headerValues[0], LABEL_HEADER_MAGIC_NUM);

  var labels = [];
  var index = headerBytes;
  while (index < buffer.byteLength) {
    var array = new Int32Array(recordBytes);
    for (var i = 0; i < recordBytes; i++) {
      array[i] = buffer.readUInt8(index++);
    }
    labels.push(array);
  }

  assert.equal(labels.length, headerValues[1]);
  return labels;
}

/** Helper class to handle loading training and test data. */
class MnistDataset {
  constructor() {
    this.dataset = null;
    this.trainSize = 0;
    this.testSize = 0;
    this.trainBatchIndex = 0;
    this.testBatchIndex = 0;
  }

  /** Loads training and test data. */
  async loadData() {
    this.dataset = await Promise.all([
      loadImages(TRAIN_IMAGES_FILE),
      loadLabels(TRAIN_LABELS_FILE),
      loadImages(TEST_IMAGES_FILE),
      loadLabels(TEST_LABELS_FILE)
    ]);
    this.trainSize = this.dataset[0].length;
    this.testSize = this.dataset[2].length;
  }

  getTrainData() {
    return this.getData_(true);
  }

  getTestData() {
    return this.getData_(false);
  }

  getData_(isTrainingData) {
    var imagesIndex;
    var labelsIndex;
    if (isTrainingData) {
      imagesIndex = 0;
      labelsIndex = 1;
    } else {
      imagesIndex = 2;
      labelsIndex = 3;
    }
    var size = this.dataset[imagesIndex].length;
    tf.util.assert(
      this.dataset[labelsIndex].length === size,
      `Mismatch in the number of images (${size}) and ` +
        `the number of labels (${this.dataset[labelsIndex].length})`
    );

    // Only create one big array to hold batch of images.
    var imagesShape = [size, IMAGE_HEIGHT, IMAGE_WIDTH, 1];
    var images = new Float32Array(tf.util.sizeFromShape(imagesShape));
    var labels = new Int32Array(tf.util.sizeFromShape([size, 1]));

    var imageOffset = 0;
    var labelOffset = 0;
    for (var i = 0; i < size; ++i) {
      images.set(this.dataset[imagesIndex][i], imageOffset);
      labels.set(this.dataset[labelsIndex][i], labelOffset);
      imageOffset += IMAGE_FLAT_SIZE;
      labelOffset += 1;
    }

    return {
      images: tf.tensor4d(images, imagesShape),
      labels: tf.oneHot(tf.tensor1d(labels, 'int32'), LABEL_FLAT_SIZE).toFloat()
    };
  }
}

In [18]:
var data;
// takes a lot of time to download files for the first time
run_async(async function () {
  data = new MnistDataset();
  await data.loadData();

  console.log(data.getTrainData());
})

  * Downloading from: https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz
  * Downloading from: https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz
  * Downloading from: https://storage.googleapis.com/cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz
  * Downloading from: https://storage.googleapis.com/cvdf-datasets/mnist/t10k-labels-idx1-ubyte.gz
{
  images: Tensor {
    kept: false,
    isDisposedInternal: false,
    shape: [ 60000, 28, 28, 1 ],
    dtype: 'float32',
    size: 47040000,
    strides: [ 784, 28, 1 ],
    dataId: {},
    id: 0,
    rankType: '4'
  },
  labels: Tensor {
    kept: false,
    isDisposedInternal: false,
    shape: [ 60000, 10 ],
    dtype: 'float32',
    size: 600000,
    strides: [ 10 ],
    dataId: {},
    id: 10,
    rankType: '2',
    scopeId: 6
  }
}


In [21]:
var tf = require('@tensorflow/tfjs-node-gpu');

var model = tf.sequential();
model.add(
  tf.layers.conv2d({
    inputShape: [28, 28, 1],
    filters: 32,
    kernelSize: 3,
    activation: 'relu'
  })
);
model.add(
  tf.layers.conv2d({
    filters: 32,
    kernelSize: 3,
    activation: 'relu'
  })
);
model.add(tf.layers.maxPooling2d({ poolSize: [2, 2] }));
model.add(
  tf.layers.conv2d({
    filters: 64,
    kernelSize: 3,
    activation: 'relu'
  })
);
model.add(
  tf.layers.conv2d({
    filters: 64,
    kernelSize: 3,
    activation: 'relu'
  })
);
model.add(tf.layers.maxPooling2d({ poolSize: [2, 2] }));
model.add(tf.layers.flatten());
model.add(tf.layers.dropout({ rate: 0.25 }));
model.add(tf.layers.dense({ units: 512, activation: 'relu' }));
model.add(tf.layers.dropout({ rate: 0.5 }));
model.add(tf.layers.dense({ units: 10, activation: 'softmax' }));

var optimizer = 'rmsprop';
model.compile({
  optimizer: optimizer,
  loss: 'categoricalCrossentropy',
  metrics: ['accuracy']
});
model.summary();

_________________________________________________________________
Layer (type)                 Output shape              Param #   
conv2d_Conv2D1 (Conv2D)      [null,26,26,32]           320       
_________________________________________________________________
conv2d_Conv2D2 (Conv2D)      [null,24,24,32]           9248      
_________________________________________________________________
max_pooling2d_MaxPooling2D1  [null,12,12,32]           0         
_________________________________________________________________
conv2d_Conv2D3 (Conv2D)      [null,10,10,64]           18496     
_________________________________________________________________
conv2d_Conv2D4 (Conv2D)      [null,8,8,64]             36928     
_________________________________________________________________
max_pooling2d_MaxPooling2D2  [null,4,4,64]             0         
_________________________________________________________________
flatten_Flatten1 (Flatten)   [null,1024]               0         
__________

In [23]:
async function run() {
  var epochs = 10;
  var batchSize = 1280;
  var { images: trainImages, labels: trainLabels } = data.getTrainData();

  var validationSplit = 0.15;

  await model.fit(trainImages, trainLabels, {
    epochs,
    batchSize,
    validationSplit,
    callbacks: {
      onEpochEnd: function (epoch, logs) {
        console.log('Epoch:', epoch, ', ', JSON.stringify(logs, null, 2));
      }
    }
  });

  var { images: testImages, labels: testLabels } = data.getTestData();
  var evalOutput = model.evaluate(testImages, testLabels);

  console.log(
    `\nEvaluation result:\n` +
      `  Loss = ${evalOutput[0].dataSync()[0].toFixed(3)}; ` +
      `Accuracy = ${evalOutput[1].dataSync()[0].toFixed(3)}`
  );

  await model.save(`file://savedModel`);
}

run_async(run);

Epoch 1 / 10


20558ms 403us/step - acc=0.693 loss=0.969 val_acc=0.898 val_loss=0.314 
Epoch: 0 ,  {
  "val_loss": 0.3143807649612427,
  "val_acc": 0.8978888988494873,
  "loss": 0.9694328308105469,
  "acc": 0.6928039193153381
}
Epoch 2 / 10


14115ms 277us/step - acc=0.929 loss=0.231 val_acc=0.936 val_loss=0.211 
Epoch: 1 ,  {
  "val_loss": 0.21135304868221283,
  "val_acc": 0.9358888864517212,
  "loss": 0.23100866377353668,
  "acc": 0.9285294413566589
}
Epoch 3 / 10


14161ms 278us/step - acc=0.960 loss=0.130 val_acc=0.976 val_loss=0.0748 
Epoch: 2 ,  {
  "val_loss": 0.07475317269563675,
  "val_acc": 0.9755555391311646,
  "loss": 0.13000273704528809,
  "acc": 0.9601960778236389
}
Epoch 4 / 10


14205ms 279us/step - acc=0.968 loss=0.0998 val_acc=0.985 val_loss=0.0522 
Epoch: 3 ,  {
  "val_loss": 0.052167169749736786,
  "val_acc": 0.9847777485847473,
  "loss": 0.0997515469789505,
  "acc": 0.9682353138923645
}
Epoch 5 / 10


...

14065ms 276us/step - acc=0.981 loss=0.0607 val_acc=0.988 val_loss=0.0434 
Epoch: 5 ,  {
  "val_loss": 0.04338466376066208,
  "val_acc": 0.988444447517395,
  "loss": 0.060730695724487305,
  "acc": 0.9805490374565125
}
Epoch 7 / 10


14114ms 277us/step - acc=0.983 loss=0.0547 val_acc=0.989 val_loss=0.0361 
Epoch: 6 ,  {
  "val_loss": 0.03607871010899544,
  "val_acc": 0.9893333315849304,
  "loss": 0.05473731458187103,
  "acc": 0.9832352995872498
}
Epoch 8 / 10


14124ms 277us/step - acc=0.985 loss=0.0462 val_acc=0.989 val_loss=0.0363 
Epoch: 7 ,  {
  "val_loss": 0.03627721220254898,
  "val_acc": 0.9894444346427917,
  "loss": 0.04619860276579857,
  "acc": 0.9854509830474854
}
Epoch 9 / 10


14132ms 277us/step - acc=0.987 loss=0.0413 val_acc=0.989 val_loss=0.0388 
Epoch: 8 ,  {
  "val_loss": 0.038812920451164246,
  "val_acc": 0.988777756690979,
  "loss": 0.04129345342516899,
  "acc": 0.987000048160553
}
Epoch 10 / 10


14220ms 279us/step - acc=0.987 loss=0.0383 val_acc=0.990 val_loss=0.0379 
Epoch: 9 ,  {
  "val_loss": 0.03786753863096237,
  "val_acc": 0.9901111125946045,
  "loss": 0.038292817771434784,
  "acc": 0.9874706268310547
}

Evaluation result:
  Loss = 0.027; Accuracy = 0.992


In [0]:
sh('zip -r savedModel.zip savedModel')

## Predict in real-time

In [0]:
var recognizer;

function predictWord() { 
  // array of words that the recognizer is trained to recognize
  const words = recognizer.wordLabels();
  recognizer.listen(({ scores }) => {
    scores = Array.from(scores).map((s, i) => ({ scores: s, word: words[1] }));
    scores.sort((s1, s2) => s2.score - s1.score);
    console.log(scores[0].word);
  }, { probabilityThreshold: 0.75 });
}

async function app() { 
  recognizer = spee
}